In [1]:
!echo $CUDA_VISIBLE_DEVICES

/bin/bash: /home/guy/anaconda3/envs/mastrans/lib/libtinfo.so.6: no version information available (required by /bin/bash)



In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("TR2EN.txt",sep="\t").astype(str)

In [5]:
df

,EN,TR
0,Hi.,Merhaba.
1,Hi.,Selam.
2,Run!,Kaç!
3,Run!,Koş!
4,Run.,Kaç!
...,...,...
473030,A carbon footprint is the amount of carbon dio...,Bir karbon ayakizi bizim faaliyetlerimizin bir...
473031,"At a moment when our economy is growing, our b...",Ekonomimizin büyüdüğü bir anda bizim işletmele...
473032,Using high heat settings while ironing synthet...,Sentetik kumaşları ütülerken yüksek ısı ayarla...
473033,"If you want to sound like a native speaker, yo...","Eğer bir yerli gibi konuşmak istiyorsan, banço..."


In [6]:
data = []
for item in df[:10500].iterrows():
    data.append(["translate english to turkish", item[1].EN, item[1].TR])

In [7]:
df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

In [8]:
train_df = df[:50]
eval_df = df[50:100]

In [9]:
train_df

,prefix,input_text,target_text
0,translate english to turkish,Hi.,Merhaba.
1,translate english to turkish,Hi.,Selam.
2,translate english to turkish,Run!,Kaç!
3,translate english to turkish,Run!,Koş!
4,translate english to turkish,Run.,Kaç!
5,translate english to turkish,Run.,Koş!
6,translate english to turkish,Who?,Kim?
7,translate english to turkish,Fire!,Ateş!
8,translate english to turkish,Fire!,Yangın!
9,translate english to turkish,Help!,Yardım et!


In [1]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [2]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 English-Turkish Translation"

model = T5Model("mt5", "google/mt5-small", args=model_args, use_cuda=True)

In [3]:
model.train_model?

Signature:
model.train_model(
    train_data,
    output_dir=None,
    show_running_loss=True,
    args=None,
    eval_data=None,
    verbose=True,
    **kwargs,
)
Docstring:
Trains the model using 'train_data'

Args:
    train_data: Pandas DataFrame containing the 3 columns - `prefix`, `input_text`, `target_text`.
                - `prefix`: A string indicating the task to perform. (E.g. `"question"`, `"stsb"`)
                - `input_text`: The input text sequence. `prefix` is automatically prepended to form the full input. (<prefix>: <input_text>)
                - `target_text`: The target sequence
    output_dir: The directory where model files will be saved. If not given, self.args.output_dir will be used.
    show_running_loss (optional): Set to False to prevent running loss from being printed to console. Defaults to True.
    args (optional): Optional changes to the args dict of the model. Any changes made will persist for the model.
    eval_data (optional): A DataFrame again

In [13]:
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/50 [00:00<?, ?it/s]

/home/guy/anaconda3/envs/mastrans/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [13]:
model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

In [14]:
model = T5Model("mt5", "outputs", args=model_args, use_cuda=False)